In [33]:
import pandas as pd
from datetime import datetime

In [95]:
# read data
fact_order = pd.read_csv('data_raw/fact_order.csv',header = 0, nrows=100)
fact_order_variant_full = pd.read_csv('data_raw/fact_order_variant_full.csv',header = 0, nrows=100)
dim_customer = pd.read_csv('data_raw/dim_customer.csv',header = 0, nrows=100)
dim_store = pd.read_csv('data_raw/dim_store.csv',header = 0, nrows=100)
dim_target_sales = pd.read_csv('data_raw/dim_target_sales.csv',header = 0, nrows=100)
dim_variant = pd.read_csv('data_raw/dim_variant.csv',header = 0, nrows=100)

For the simplicity of the project, we will only try to aggregate the sales data by date and store. Thus the column to use in fact_order will be `order_id; created_at; store_id; currency; fulfillment_status; pos_date`

In [98]:
# read data
usecols = ['order_id','total_price','created_at', 'store_id', 'currency', 'fulfillment_status', 'pos_date']

dtype_dict = {
    'store_id':'category',
    'currency':'category',
    'fulfillment_status':'category',
}

fact_order = pd.read_csv('data_raw/fact_order.csv',header = 0, usecols=usecols, dtype=dtype_dict)

In [99]:
# data frame info
fact_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1510893 entries, 0 to 1510892
Data columns (total 7 columns):
 #   Column              Non-Null Count    Dtype   
---  ------              --------------    -----   
 0   order_id            1510893 non-null  int64   
 1   created_at          1510893 non-null  object  
 2   store_id            1510866 non-null  category
 3   currency            1510866 non-null  category
 4   total_price         1510866 non-null  float64 
 5   fulfillment_status  1510866 non-null  category
 6   pos_date            1468380 non-null  object  
dtypes: category(3), float64(1), int64(1), object(2)
memory usage: 50.4+ MB


In [115]:
def processing(df, save_to_csv = False):
    df = df.query('fulfillment_status == "fulfilled"')     # only keep filfilled orders:
    df.created_at = df.created_at.map(lambda x:x[:10])  # get only date data
    df.created_at = pd.to_datetime(df['created_at'], format="%Y-%m-%d") 
    df = df.groupby(['created_at','store_id']).sum()['total_price'].reset_index()
    df.total_price.fillna(0,inplace=True)
    df.total_price = df.total_price.round(0).astype('int64') # round and convert sales to interger 
    df.rename({'total_price':'sales'},axis = 1,inplace = True)
    if save_to_csv:
        df.to_csv("agg_sales_store_daily.csv",index = False)
    return df

In [118]:
df = fact_order.copy()
processing(df, save_to_csv = True)

C:\Users\Admin\anaconda3\envs\DataMining_Lab\lib\site-packages\pandas\core\generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,created_at,store_id,sales
0,2017-05-19,295646,0
1,2017-05-19,307222,4950400
2,2017-05-19,307244,0
3,2017-05-19,307248,0
4,2017-05-19,320264,0
...,...,...,...
128565,2021-08-24,947306,0
128566,2021-08-24,947302,0
128567,2021-08-24,947298,0
128568,2021-08-24,1005758,2027700
